In [3]:
import tensorflow as tf
import argparse
import os
from config import cfg
from easydict import EasyDict
from utils.kitti_loader import iterate_data
from model.group_pointcloud import FeatureNet
import numpy as np
from model.rpn_1 import MiddleAndRPN

In [4]:
parser=EasyDict()
parser.i = 1
parser.tag = 'Test1'
parser.single_batch_size = 2
parser.lr =0.001
parser.al =1
parser.output_path = './prediction'
parser.v=False

dataset_dir = cfg.DATA_DIR
train_dir = os.path.join(cfg.DATA_DIR, 'training')
val_dir = os.path.join(cfg.DATA_DIR, 'validation')
log_dir = os.path.join('./log', parser.tag)
save_model_dir = os.path.join('./save_model', parser.tag)
os.makedirs(log_dir, exist_ok=True)
os.makedirs(save_model_dir, exist_ok=True)

In [5]:
batches=iterate_data(train_dir,batch_size=2)
#tag,labels,vox_feature,vox_number,vox_coordinate,rgb,raw_lidar
batch=next(batches)
voxel_feature=batch[2]
voxel_coordinate=batch[4]

In [6]:
A= tf.convert_to_tensor(voxel_coordinate)

In [7]:
A=tf.Variable([1.,2.,3.])
A

<tf.Variable 'Variable:0' shape=(3,) dtype=float32, numpy=array([1., 2., 3.], dtype=float32)>

In [8]:
tf.slice(A,[0],[1])

<tf.Tensor: id=13, shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>

In [9]:
type(np.int32(voxel_coordinate.shape[1]))==np.int32

True

In [10]:
print("{}:{}".format("tag",batch[0].dtype))
print("{}:{}".format("labels",batch[1].dtype))
print("{}:{}".format("vox_feature",batch[2].dtype))
print("{}:{}".format("vox_number",batch[3].dtype))
print("{}:{}".format("vox_coordinate",batch[4].dtype))
print("{}:{}".format("rgb",batch[5].dtype))
print("{}:{}".format("raw_lidar",batch[6][0].dtype))

tag:<U6
labels:object
vox_feature:float32
vox_number:int64
vox_coordinate:int64
rgb:uint8
raw_lidar:float32


In [11]:
A=FeatureNet(voxel_feature[0],voxel_coordinate[0],training=True,batch_size=2)
output=A.outputs
output.shape

TensorShape([2, 10, 400, 352, 128])

In [12]:
from utils.utils import *
anchors = cal_anchors()
pos_equal_one, neg_equal_one, targets = cal_rpn_target(batch[1], [cfg.FEATURE_HEIGHT,cfg.FEATURE_WIDTH], anchors)
pos_equal_one[..., [0]].shape
pos_equal_one_for_reg = np.concatenate([np.tile(pos_equal_one[..., [0]], 7), np.tile(pos_equal_one[..., [1]], 7)], axis=-1)
pos_equal_one_sum = np.clip(np.sum(pos_equal_one, axis=(1, 2, 3)).reshape(-1, 1, 1, 1), a_min=1, a_max=None)
neg_equal_one_sum = np.clip(np.sum(neg_equal_one, axis=(1, 2, 3)).reshape(-1, 1, 1, 1), a_min=1, a_max=None)
info={"pos_equal_one":pos_equal_one,"neg_equal_one":neg_equal_one,"targets":tf.cast(targets,tf.float32),
      "pos_equal_one_for_reg":pos_equal_one_for_reg,"pos_equal_one_sum":pos_equal_one_sum,
      "neg_equal_one_sum":neg_equal_one_sum}

In [13]:
Model=MiddleAndRPN()
model=Model(output,info)

v=model.trainable_variables

In [14]:
print(model.pos_equal_one.shape)
model.cls_neg_loss.shape
neg_equal_one_sum

(2, 200, 176, 2)


array([[[[70400.]]],


       [[[70374.]]]])

In [15]:
tf.convert_to_tensor(neg_equal_one.shape)

<tf.Tensor: id=2393, shape=(4,), dtype=int32, numpy=array([  2, 200, 176,   2], dtype=int32)>

In [16]:
def average_gradients(tower_grads):
    # ref:
    # https://github.com/tensorflow/models/blob/6db9f0282e2ab12795628de6200670892a8ad6ba/tutorials/image/cifar10/cifar10_multi_gpu_train.py#L103
    # but only contains grads, no vars
    average_grads = []
    for grad_and_vars in zip(*tower_grads):
        grads = []
        for g in grad_and_vars:
            # Add 0 dimension to the gradients to represent the tower.
            expanded_g = tf.expand_dims(g, 0)

            # Append on a 'tower' dimension which we will average over below.
            grads.append(expanded_g)

        # Average over the 'tower' dimension.
        grad = tf.concat(axis=0, values=grads)
        grad = tf.reduce_mean(input_tensor=grad, axis=0)

        # Keep in mind that the Variables are redundant because they are shared
        # across towers. So .. we will just return the first tower's pointer to
        # the Variable.
        grad_and_var = grad
        average_grads.append(grad_and_var)
    return average_grads

In [17]:
batch = next(iterate_data(train_dir,batch_size=2,multi_gpu_sum=1))
#batches = iterate_data(train_dir)
#batch = next(batches)
tag = batch[0]
labels = batch[1]
voxel_feature = batch[2]
vox_number = batch[3] # original is int64
voxel_coordinate =batch[4] # original is int64
rgb = batch[5]
raw_lidar = batch[6]

In [25]:
tf.config.set_soft_device_placement(True)
class RPN3D(object):
    def __init__(self,cls="Car",single_batch_size=1,learning_rate=0.001,max_gradient_norm=5.0,
                 alpha=1.5,beta=1,training=False):
        self.cls = cls
        self.single_batch_size = single_batch_size
        self.learning_rate = tf.Variable(float(learning_rate), trainable=False, dtype=tf.float32)
        self.global_step = tf.Variable(1, trainable=False)
        self.epoch = tf.Variable(0, trainable=False)
        self.epoch_add_op = self.epoch.assign(self.epoch + 1)
        self.alpha = alpha
        self.max_gradient_norm = max_gradient_norm
        self.beta = beta
         
        boundaries = [80, 120]
        values = [ self.learning_rate, self.learning_rate * 0.1, self.learning_rate * 0.01 ]
        self.lr = tf.compat.v1.train.piecewise_constant(self.epoch, boundaries, values)

        #build graph
        self.is_train=training
        
        self.vox_feature = []
        self.vox_number = []
        self.vox_coordinate = []
        self.targets = []
        self.pos_equal_one = []
        self.pos_equal_one_sum = []
        self.pos_equal_one_for_reg = []
        self.neg_equal_one = []
        self.neg_equal_one_sum = []
        
        self.delta_output = []
        self.prob_output = []
        self.opt = tf.keras.optimizers.Adam(learning_rate=self.lr)
        self.gradient_norm = []
        self.tower_grads = []
        self.anchors = cal_anchors()
        self.rpn = MiddleAndRPN()
        
    def __call__(self,train_dir):

        for idx, batch in enumerate(iterate_data(train_dir,batch_size=1,multi_gpu_sum=1)):
            if idx==4:
                return None
            else:
                #batches = iterate_data(train_dir)
                #batch = next(batches)
                tag = batch[0]
                labels = batch[1]
                voxel_feature = batch[2]
                vox_number = tf.cast(batch[3],dtype=tf.int32) # original is int64
                voxel_coordinate =tf.cast(batch[4],dtype=tf.int32) # original is int64
                rgb = batch[5]
                raw_lidar = batch[6]
                # get from labels
                pos_equal_one, neg_equal_one, targets = cal_rpn_target(labels, [cfg.FEATURE_HEIGHT,cfg.FEATURE_WIDTH], self.anchors)
                pos_equal_one_for_reg = np.concatenate([np.tile(pos_equal_one[..., [0]], 7), np.tile(pos_equal_one[..., [1]], 7)], axis=-1)
                pos_equal_one_sum = np.clip(np.sum(pos_equal_one, axis=(1, 2, 3)).reshape(-1, 1, 1, 1), a_min=1, a_max=None)
                neg_equal_one_sum = np.clip(np.sum(neg_equal_one, axis=(1, 2, 3)).reshape(-1, 1, 1, 1), a_min=1, a_max=None)
                info={"pos_equal_one":pos_equal_one,"neg_equal_one":neg_equal_one,"targets":tf.cast(targets,dtype=tf.float32),
                      "pos_equal_one_for_reg":pos_equal_one_for_reg,"pos_equal_one_sum":pos_equal_one_sum,
                      "neg_equal_one_sum":neg_equal_one_sum}

                # model running
                with tf.GradientTape() as tape:
                    # feature is initial address for saving class FeatureNet
                    feature=FeatureNet(voxel_feature[0],voxel_coordinate[0],training=self.is_train,batch_size=1)
                    rpn=self.rpn(inputs=feature.outputs,info=info,alpha=self.alpha, beta=self.beta, training=self.is_train)
                print("{}:{}".format("loss",rpn.loss))
                
                # list add A + B
                #self.params = rpn.variables+feature.variables
                self.trainable_params = rpn.trainable_variables+feature.trainable_variables
                
                # output
                feature_output = feature.outputs
                delta_output = rpn.delta_output
                prob_output = rpn.prob_output

                # loss and grad
                self.loss = rpn.loss
                self.reg_loss = rpn.reg_loss
                self.cls_loss = rpn.cls_loss
                self.cls_pos_loss = rpn.cls_pos_loss_rec
                self.cls_neg_loss = rpn.cls_neg_loss_rec
                
                #update the parametr
                # list add A + B
                #self.params = rpn.variables+feature.variables
                self.trainable_params = rpn.trainable_variables+feature.trainable_variables
                gradients = tape.gradient(rpn.loss,self.trainable_params)
                # clip the gradients
                clipped_gradients, gradient_norm = tf.clip_by_global_norm(gradients, self.max_gradient_norm)
                self.opt.apply_gradients(zip(gradients,self.trainable_params))
                
                self.delta_output=delta_output
                self.prob_output=prob_output
                #self.tower_grads.append(clipped_gradients)
                #self.gradient_norm.append(gradient_norm)
                self.rpn_output_shape= rpn.output_shapes





In [26]:
model=RPN3D()
model(train_dir)

loss:0.7796424627304077
loss:3.8272995948791504
loss:59.21971130371094
loss:60.26079559326172


In [ ]:
i=0
# batch: (tag,labels,vox_feature,vox_number,vox_coordinate,rgb,raw_lidar)
for batch in iterate_data(train_dir):
    i+=1
    if i == 2:
        break
    else:
        pass
        #print("batch:",batch[2])
        # build model
        #input data
